In [3]:
import yfinance as yf
import talib
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)  # Ignore DeprecationWarning
warnings.filterwarnings('ignore', category=FutureWarning) 

# Function to fetch historical stock data using yfinance
def fetch_stock_data(ticker, start_date, end_date):
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        return stock_data
    except: 
        print("error in fetching data")
# Function to calculate MACD using TA-Lib
def calculate_macd(data,fast_period,slow_period):
    try:
        if(fast_period > slow_period):
            temp = fast_period
            fast_period = slow_period
            slow_period = temp
        if(fast_period>1):
            macd, signal, _ = talib.MACD(data['Close'], fastperiod=fast_period, slowperiod=slow_period, signalperiod=9)
        else: 
            fast_period = 2 
            macd, signal, _ = talib.MACD(data['Close'], fastperiod=fast_period, slowperiod=slow_period, signalperiod=9)
            
        return macd, signal
    except: 
        print(f"error in calculatin macd signal {fast_period,slow_period}")
# Function to calculate Rate of Change (ROC) using TA-Lib
def calculate_roc(data, period):
    try:
        roc = talib.ROC(data['Close'], timeperiod=period)
        return roc
    except:
        print("this period caused the error",period)


# Function to calculate the derivative of RSI
# Function to calculate the derivative of RSI
def calculate_rsi_derivative(data, period):
    try:
        if len(data) >= period:
            rsi = talib.RSI(data['Close'], timeperiod=period)
            rsi_derivative = data.Close.diff()
            return rsi_derivative
        else:
            print("Insufficient data for RSI derivative calculation")
            return None
    except Exception as e:
        print(f"Error calculating RSI derivative: {e}")
        print(period)
        return None
# Function to simulate a trailing stop-loss exit condition
def trailing_stop_loss(data, stop_loss_percentage):
    highest_high = data['High'].rolling(window=20).max()
    stop_loss = highest_high * (1 - stop_loss_percentage)
    return stop_loss

In [22]:
def evaluate_strategy(data, fast_period, slow_period, fast_p_exit, slow_p_exit,roc_period, rsi_period, roc_threshold, rsi_derivative_threshold, stop_loss_percentage):
    # Calculate indicators
    print(fast_period,slow_period,fast_p_exit, slow_p_exit,roc_period, rsi_period, roc_threshold, rsi_derivative_threshold, stop_loss_percentage)
    data['MACD'], data['Signal'] = calculate_macd(data, fast_period, slow_period)
    data['MACD_E'], data['Signal_E'] = calculate_macd(data, fast_p_exit, slow_p_exit)

    data['ROC'] = calculate_roc(data, roc_period)
    data['RSI_Derivative'] = calculate_rsi_derivative(data, rsi_period)
    
    # Apply trailing stop-loss exit condition
#     stop_loss = trailing_stop_loss(data, stop_loss_percentage)
    
    # Simulated trading logic
    position = 0  # 0 for no position, 1 for long position
    profit = 0
    total_trades = 0
    
    stop_loss_price = 0
    
    for i in range(1, len(data)):
        try:
            if data['MACD'][i] > data['Signal'][i]:
                if position == 0:
                    position = 1  # Buy (enter long position)
                    entry_price = data['Close'][i]
                    stop_loss_price = entry_price - (entry_price * 0.01 * stop_loss_percentage)
                    print(f"Entry: {entry_price} on {data.index[i]} Stop Loss: {stop_loss_price}")
                    total_trades += 1  
            elif position == 1:
                if data['Close'][i] > entry_price:  # Adjust stop loss only when the price goes up
                    stop_loss_price = max(stop_loss_price, data['Close'][i] - (data['Close'][i] * 0.01 * stop_loss_percentage))

                if data['Low'][i] < stop_loss_price:
                    position = 0  # Sell due to stop loss
                    exit_price = data['Open'][i] if stop_loss_price > data['Open'][i] else stop_loss_price
                    profit += exit_price - entry_price  # Calculate profit/loss
                    print(f"Exit: {exit_price} on {data.index[i]} exited on stop loss")
                    print(f"Profit: {profit}")
                elif data['MACD_E'][i] < data['Signal_E'][i]:
                    position = 0  # Sell due to exit condition
                    exit_price = data['Close'][i]
                    profit += exit_price - entry_price  # Calculate profit/loss
                    print(f"Exit: {exit_price} on {data.index[i]} exited on MACD condition")
                    print(f"Profit: {profit}")
        except:
            print("Something caused the error")
    
    return profit

In [8]:
def generate_offspring(elite):
    new_population = []
    elite_size = len(elite)
    
    # Create new individuals by performing uniform crossover and mutation
    while len(new_population) < population_size:
        parent1 = random.choice(elite)
        parent2 = random.choice(elite)
        
        # Perform uniform crossover
        child = uniform_crossover(parent1, parent2)
        
        # Apply mutation
        if random.random() < mutation_rate:
            child = mutate(child)
        
        new_population.append(child)
    
    return new_population

def uniform_crossover(parent1, parent2):
    child = []
    for gene1, gene2 in zip(parent1, parent2):
        # Randomly select genes from parents for the child
        if random.choice([True, False]):
            child.append(gene1)
        else:
            child.append(gene2)
    return child

def mutate(individual):
    # Implement mutation logic here
    # ...
    return individual


In [2]:
# Fetch historical stock data
ticker_symbol = '^NSEI'  # Replace with your desired stock symbol
start_date = '2021-01-01'
end_date = '2022-12-31'
stock_data = fetch_stock_data(ticker_symbol, start_date, end_date)

# Genetic Algorithm parameters
population_size = 500
num_generations = 20
mutation_rate = 0.1
tournament_size = 5 # no longer used/
elite_size = 200

# GA: Initialization
def generate_individual():
    fast_p = random.randint(2,10)
    slow_p = random.randint(fast_p,20)
    fast_p_exit = random.randint(2,10)
    slow_p_exit = random.randint(fast_p_exit,20)
    return [fast_p,slow_p,fast_p_exit,slow_p_exit ,random.randint(1, 7), random.randint(2, 7), random.randint(1, 30), random.randint(1, 30), random.randint(0,3)]

def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

def mutate(individual):
    gene_to_mutate = random.randint(0, len(individual) - 1)
    if gene_to_mutate < 2:
#         individual[gene_to_mutate] = random.randint(5, 200)
        pass
    else:
        individual[gene_to_mutate] = random.randint(1, 5) if gene_to_mutate == 5 else random.randint(1, 30)
    return individual
# Known or useful parameter values
known_parameters = [
    [3, 15, 5, 5, 15, 20, 2,3,15],  # Example set of known parameters
    # Add more known parameter sets if available
]

# Randomly generate the rest of the population
remaining_population = [generate_individual() for _ in range(population_size - len(known_parameters))]

# Initialize population using known parameter values and randomly generated values
population = known_parameters + remaining_population
# population = [generate_individual() for _ in range(population_size)]

# GA: Evolution process
for generation in range(num_generations):
    fitness_scores = [evaluate_strategy(stock_data.copy(), *individual) for individual in population]
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]
    
    new_population = []
    
    for generation in range(num_generations):
        fitness_scores = [evaluate_strategy(stock_data.copy(), *individual) for individual in population]
        sorted_population = [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]

        # Select the top K individuals (elite) for the next generation
        elite = sorted_population[:elite_size]

        # Check for convergence or other stopping criteria

        # If convergence or stopping condition met, break the loop
#         if stopping_condition_met:
#             break

        # Create new population by generating offspring from the elite individuals
        new_population = generate_offspring(elite)

        population = new_population
    
#     for _ in range(population_size):
#         tournament_indices = random.sample(range(population_size), tournament_size)
#         tournament_contestants = [sorted_population[i] for i in tournament_indices]
#         parent1, parent2 = random.sample(tournament_contestants, 2)  # Select two parents
        
#         child1 = crossover(parent1, parent2)  # Apply crossover
#         child2 = crossover(parent2, parent1)  # Apply crossover with parents swapped
        
#         # Perform mutation on children
#         if random.random() < mutation_rate:
#             child1 = mutate(child1)
#         if random.random() < mutation_rate:
#             child2 = mutate(child2)
        
#         new_population.extend([child1, child2])  # Add children to the new population
    
#     population = new_population[:population_size]  # Select the best individuals to form the next generation
        
#         winner = max(tournament_contestants, key=lambda x: evaluate_strategy(stock_data.copy(), *x))
        
#         if random.random() < mutation_rate:
#             winner = mutate(winner)
        
#         new_population.append(winner)
    
#     population = new_population

# Find the best individual (parameters) from the final population
best_individual_index = np.argmax(fitness_scores)
best_individual = population[best_individual_index]
best_fitness_score = fitness_scores[best_individual_index]

print("Best Parameters:", best_individual)
print("Best Fitness Score:", best_fitness_score)


error in fetching data


NameError: name 'evaluate_strategy' is not defined

In [23]:
stock_data = yf.download("^NSEI", start='2023-01-01',end='2023-12-28')

[*********************100%%**********************]  1 of 1 completed


In [31]:
# Use the best parameters obtained from the GA
#"Best Parameters (Fast Period, Slow Period, ROC Period, RSI Period, ROC Threshold, RSI Derivative Threshold, Stop Loss %):"
best_parameters = [2, 3, 4,16, 6, 4, 9, 12, 0.1]

# Apply the strategy on out-of-sample data using the best parameters
out_of_sample_profit = evaluate_strategy(stock_data.copy(), *best_parameters)


# Print or analyze the performance metrics
print("Out-of-sample profit:", out_of_sample_profit)

2 3 4 16 6 4 9 12 0.1
Entry: 17894.849609375 on 2023-01-16 00:00:00 Stop Loss: 17876.954759765624
Exit: 18009.622740234376 on 2023-01-25 00:00:00 exited on stop loss
Profit: 114.77313085937567
Entry: 17616.30078125 on 2023-02-01 00:00:00 Stop Loss: 17598.68448046875
Exit: 17753.129490234376 on 2023-02-13 00:00:00 exited on stop loss
Profit: 251.6018398437518
Entry: 17929.849609375 on 2023-02-14 00:00:00 Stop Loss: 17911.919759765624
Exit: 17926.25501953125 on 2023-02-17 00:00:00 exited on stop loss
Profit: 248.0072500000024
Entry: 17450.900390625 on 2023-03-01 00:00:00 Stop Loss: 17433.449490234376
Exit: 17443.80078125 on 2023-03-10 00:00:00 exited on stop loss
Profit: 240.9076406250024
Entry: 17100.05078125 on 2023-03-17 00:00:00 Stop Loss: 17082.95073046875
Exit: 17076.19921875 on 2023-03-24 00:00:00 exited on stop loss
Profit: 217.0560781250024
Entry: 17080.69921875 on 2023-03-29 00:00:00 Stop Loss: 17063.61851953125
Exit: 17807.30078125 on 2023-04-13 00:00:00 exited on stop loss
Pr

In [58]:
elite

[[9, 6, 30, 21, 3, 4, 11, 27, 1],
 [9, 6, 30, 21, 29, 5, 2, 27, 1],
 [9, 6, 30, 21, 22, 2, 11, 24, 1],
 [9, 6, 30, 21, 29, 7, 15, 12, 1],
 [9, 6, 30, 21, 3, 5, 11, 12, 1],
 [9, 6, 30, 21, 3, 7, 20, 24, 1],
 [9, 6, 30, 21, 3, 4, 16, 27, 1],
 [9, 6, 30, 21, 2, 5, 11, 23, 1],
 [9, 6, 30, 21, 26, 2, 11, 12, 1],
 [9, 6, 30, 21, 3, 7, 11, 7, 1],
 [9, 6, 30, 21, 14, 2, 11, 12, 1],
 [9, 6, 30, 21, 3, 7, 11, 8, 1],
 [9, 6, 30, 21, 3, 4, 28, 24, 1],
 [9, 6, 30, 21, 26, 7, 11, 4, 1],
 [9, 6, 30, 21, 30, 7, 11, 9, 1],
 [9, 6, 30, 21, 29, 2, 11, 24, 1],
 [9, 6, 30, 21, 4, 7, 11, 12, 1],
 [9, 6, 30, 21, 30, 7, 11, 12, 1],
 [9, 6, 30, 21, 25, 5, 11, 12, 1],
 [9, 6, 30, 21, 29, 2, 11, 4, 1],
 [9, 6, 30, 21, 26, 7, 11, 4, 1],
 [9, 6, 30, 21, 30, 4, 11, 7, 1],
 [9, 6, 30, 21, 3, 5, 11, 21, 1],
 [9, 6, 30, 21, 1, 7, 2, 24, 1],
 [9, 6, 30, 21, 29, 5, 11, 24, 1],
 [9, 6, 30, 21, 30, 7, 28, 8, 1],
 [9, 6, 30, 21, 25, 5, 11, 8, 1],
 [9, 6, 30, 21, 3, 2, 2, 9, 1],
 [9, 6, 30, 21, 4, 7, 15, 4, 1],
 [9, 6, 30, 

In [9]:
len(population)

100

In [10]:
population

[[9, 20, 1, 5, 24, 9, 3],
 [9, 20, 1, 5, 24, 9, 22],
 [6, 13, 1, 5, 19, 13, 1],
 [6, 13, 1, 7, 12, 12, 1],
 [3, 12, 3, 5, 12, 9, 3],
 [3, 12, 3, 5, 12, 22, 3],
 [9, 10, 24, 7, 24, 9, 3],
 [3, 20, 24, 4, 5, 12, 24],
 [9, 19, 7, 2, 24, 1, 1],
 [2, 19, 1, 7, 19, 12, 3],
 [3, 20, 1, 7, 24, 4, 1],
 [3, 20, 7, 2, 3, 11, 1],
 [9, 20, 6, 7, 12, 12, 3],
 [6, 20, 3, 7, 12, 3, 3],
 [3, 19, 5, 7, 19, 12, 1],
 [9, 10, 24, 4, 5, 12, 1],
 [3, 19, 5, 7, 19, 12, 1],
 [7, 19, 7, 7, 19, 12, 1],
 [2, 20, 1, 3, 19, 3, 2],
 [3, 6, 1, 3, 19, 9, 2],
 [2, 20, 3, 7, 24, 9, 3],
 [6, 20, 3, 12, 24, 28, 24],
 [9, 19, 1, 5, 12, 1, 20],
 [3, 6, 1, 5, 12, 9, 20],
 [3, 12, 6, 6, 12, 3, 3],
 [9, 20, 6, 6, 12, 3, 1],
 [7, 20, 3, 7, 24, 9, 1],
 [7, 20, 3, 7, 24, 29, 1],
 [3, 20, 1, 7, 24, 28, 24],
 [2, 6, 1, 7, 24, 9, 3],
 [3, 19, 7, 2, 3, 11, 1],
 [3, 19, 7, 2, 3, 11, 1],
 [6, 12, 1, 5, 12, 9, 20],
 [3, 12, 8, 5, 12, 9, 1],
 [6, 20, 3, 7, 24, 9, 16],
 [7, 19, 3, 7, 24, 5, 3],
 [4, 19, 4, 7, 24, 9, 2],
 [3, 6, 4, 7, 24, 